# Introduction

When passing text data to a LLM, either for a summarisation task or to create a vector database for RAG applications, the given text might be too long for the LLM to handle in one call. There are usually two avenues to go about this issue -- chunking and more powerful LLMs.

While great progress has been made in improving LLMs' capability of handling longer context window, splitting text into smaller chunks is still the common approach widely used in many LLM applications. 

In this notebook, I'd like to experiment different chunking strategies to see how it works with my PaperSummariser. 

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from llmsherpa.readers import LayoutPDFReader
import sys
from pathlib import Path

print(sys.path)
sys.path.append('/mnt/d/Projects/papersurvey_tool/src/example_package/')

['/mnt/d/Projects/papersurvey_tool/notebooks', '/python', '/mnt/d/Projects/papersurvey_tool/notebooks', '/home/alexis/.pyenv/versions/3.10.12/lib/python310.zip', '/home/alexis/.pyenv/versions/3.10.12/lib/python3.10', '/home/alexis/.pyenv/versions/3.10.12/lib/python3.10/lib-dynload', '', '/mnt/d/Projects/papersurvey_tool/.venv/lib/python3.10/site-packages', '/home/alexis/.cache/pdm/packages/rouge-1.0.1-py3-none-any/lib', '/home/alexis/.cache/pdm/packages/torchvision-0.15.1-cp310-cp310-manylinux1_x86_64/lib']


In [2]:
OPENAI_API_KEY  = os.environ["OPENAI_API_KEY"]
GOOGLE_SEARCH_API_KEY = os.environ["GOOGLE_SEARCH_API_KEY"]

In [35]:
import prompts

default_prompts = {"system_prompt": prompts.ROLE_PROMPT,
                   "user_prompt": prompts.INSTRUCTION_PROMPT,
                   "extraction_prompt": prompts.EXTRACTION_PROMPT}

In [36]:
def summarise_page(text_chunk, default_prompts):
        
        prompt = default_prompts["user_prompt"].format(text_chunk=text_chunk)
        messages = [{"role": "system", "content": default_prompts["system_prompt"]},
                {"role": "user", "content": prompt}]
        client = OpenAI()
        response = client.chat.completions.create(
                        messages=messages,
                        model = "gpt-4"
        )
        return response.choices[0].message.content

## split_by_page

In [43]:
def split_by_page(file_path):

    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=0)
    text_chunks = text_splitter.split_documents(documents)

    return text_chunks


## split by chunk size

Splitting by text chunk size is straightforward and can be easily done with Langchain splitter, with the limitation that it does not account for references and appendices which not useful for the summarisation task. Hard coded selection could be used to address this, as I did in the page-based splitting method. 

In [17]:
def split_by_chunk_size(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    content = "\n".join([doc.page_content for doc in documents])
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
    text_chunks = text_splitter.split_text(content)
    return text_chunks

## split by layout

In [6]:
def split_by_section(file_path):

    llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
    # pdf_url = "../example_paper1.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
    pdf_reader = LayoutPDFReader(llmsherpa_api_url)
    doc = pdf_reader.read_pdf(file_path)
    
    section_list = [section for section in doc.sections() if section.level == 0]
    remove_list = ["references", "appendix", "acknowledgment"]
    selected_sections = [section for section in section_list if not any(remove_section in section.title.lower() for remove_section in remove_list)]

    text_chunks = []
    for i, section in enumerate(selected_sections):
        text = section.to_text(include_children=True, recurse=True)
        if len(text) > 200:
            text_to_include = ""
            text_to_include += text
            text_chunks.append(text_to_include)

    return text_chunks
        

I decided to use the one below as it splits by sections and is more suitable given many academic papers follow similar section structure. 

First, let's see what sections it identifies for the example article and the cooresponding section level. 0 is the highest level and higher numbers indicate sublevels. 

In [7]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "../example_paper1.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)
text_chunks = []
for section in doc.sections(): 
    print((section.level, section.title))

(0, 'Why do tree-based models still outperform deep learning on tabular data?')
(0, 'Léo Grinsztajn Soda, Inria Saclay')
(0, '1 Introduction')
(0, '2 Related work')
(0, '3 A benchmark for tabular learning')
(1, '3.1 45 reference tabular datasets')
(1, '3.2 Removing side issues')
(1, '3.3 A procedure to benchmark models with hyperparameter selection')
(1, '3.4 Aggregating results across datasets')
(1, '3.5 Data preparation')
(0, '4 Tree-based models still outperform deep learning on tabular data.')
(1, '4.1 Models benchmarked')
(1, '4.2 Results')
(0, '5 Empirical investigation: why do tree-based models still outperform deep')
(0, 'learning on tabular data?')
(0, '6 Discussion and conclusion')
(0, 'Acknowledgments and Disclosure of Funding')
(0, 'References')
(1, 'Y. Chen')
(1, 'I. D')
(0, 'A Appendix')
(1, 'A.1.1 Numerical classiﬁcation')
(1, 'A.1.2 Numerical regression')
(1, 'A.1.3 Categorical classiﬁcation')
(1, 'A.1.4 Categorical regression')
(1, 'A.2 More benchmarks')
(1, 'Ỹ (Xi) =

It seems to work fine with some limitations identified. 
- The title paper and author names are extracted as top-level sections.
- If a section title is too long and separated in two lines, e.g. section 5, it recognised them as two sections. 
- Because the full paper is parsed, some sections in appendices are not correctly recognised. 

Since I only care about the main body of the paper for the summarisation task, I decided to remove the irrelavent parts such as reference, appendix and acknowledgments. This leaves in total 9 sections with the title and incomplete section title remaining as individual sections. My expectation was that the model should be able "realise" this is not a section and tell me it cannot provide a summary for just one line sentence, but I was wrong. 

In [9]:
section_list = [section for section in doc.sections() if section.level == 0]
remove_list = ["references", "appendix", "acknowledgment"]
remove_list = ["references", "appendix"]
selected_sections = [section for section in section_list if not any(remove_section in section.title.lower() for remove_section in remove_list)]
print([section.title for section in selected_sections])

['Why do tree-based models still outperform deep learning on tabular data?', 'Léo Grinsztajn Soda, Inria Saclay', '1 Introduction', '2 Related work', '3 A benchmark for tabular learning', '4 Tree-based models still outperform deep learning on tabular data.', '5 Empirical investigation: why do tree-based models still outperform deep', 'learning on tabular data?', '6 Discussion and conclusion', 'Acknowledgments and Disclosure of Funding']


In [37]:
text_chunks_summary = []
for section in selected_sections:
    text_chunk = section.to_text(include_children=True, recurse=True)
    page_summary = summarise_page(text_chunk, default_prompts)
    text_chunks_summary.append(page_summary)
print(len(text_chunks_summary))

10


I initially chose gpt-3.5 due to its low cost, but find **gpt-3.5 writes something somewhat relevant to the topic but mostly fabricated when only the paper title is provided as context.**

> 'Summary: The paper investigates why tree-based models still outperform deep learning models on tabular data. The authors highlight that while deep learning has been successful in various domains, it struggles with tabular data due to its inherent characteristics. They identify several reasons for this, including the requirement for more data and the inefficiency of deep learning in modeling interactions between features. The paper also explores the importance of interpretability and explainability in tabular data analysis, which tree-based models provide but deep learning models do not.\n\nFindings:\n- Deep learning models struggle with tabular data due to factors such as the need for more data and their inefficiency in capturing feature interactions.\n- Tree-based models outperform deep learning models on tabular data due to their ability to incorporate domain knowledge and provide interpretability.\n- Deep learning shows promise in certain domains, but more research is needed to improve its performance on tabular data.\n\nMethods:\nThe authors conducted a thorough review of existing literature and research studies to analyze the performance of tree-based models and deep learning models on tabular data. They identified key factors that contribute to the performance differences between these models and discussed the limitations of deep learning in this specific context. The paper also examined the importance of interpretability and explainability in tabular data analysis, highlighting the advantages that tree-based models offer in these areas.'

After changing the model to gpt-4 and keeping everythin else the same, the output appears more reasonable.

> 'Sorry, I am not able to provide the summary, findings, and methods as there is not enough information given. You only provided the title of a research paper, not a piece of text from it. The title itself, "Why do tree-based models still outperform deep learning on tabular data?" unfortunately does not contain any conclusions, findings, or methods for me to summarize or list out. Please provide a specific excerpt or paragraph from the research paper so I can help you summarize it.'

Because of this hallucination problem of gpt-3.5, I will use gpt-4 for next steps. 


In [28]:
text_chunks_summary[0]

'I\'m sorry, I can\'t provide the summary, findings, or methods since you didn\'t provide any details from the research paper, "Why do tree-based models still outperform deep learning on tabular data?". I need specific information from the paper to generate accurate and concise information based on it. Please upload some details from the paper for me to help you.'

In [38]:
# glimpse of section-level summaries
for i, text in enumerate(text_chunks_summary):
    print((i, text[:100]))

(0, "I'm sorry, but it would be hard for me to summarise the research paper as you've only given me the t")
(1, 'Summary: The study revealed that tree-based models, including XGBoost and Random Forests, remain the')
(2, 'Summary: The research underscores that tree-based models remain superior to deep learning models in ')
(3, 'Summary: The literature reveals attempts to enable deep learning on tabular data by data encoding, h')
(4, 'Summary: The research provides a robust benchmark for evaluating machine learning models on tabular ')
(5, 'Summary: The research paper establishes that tree-based models significantly outperform deep learnin')
(6, "I'm sorry, but I can't provide the summary, findings, and methods as you have not provided any speci")
(7, 'Summary: The research demonstrates that tree-based models outperform Neural Networks (NN) on tabular')
(8, 'Summary: The research concludes that tree-based models often perform better than NNs on tabular data')
(9, 'Summary: The report ack

The next step is to stitch all section-level summary and ask the model to generate a final summary without changing the prompts. The hope is that the model will only use valid summaries and ignore sentences that says "Sorry, I am not able to provide the summary, ..." as these sentences constitute a small part of the input. However, this is not always the case. 

In particular, the model is more likely to fail the task (sayinng `I'm sorry, but you didn't provide a piece of text that I should summarise. Could you please share the text you want summarised? I will then provide a summary of the key conclusions, list the relevant findings, and describe the methods as per your instructions.`) when:

- there is insufficient relevant information in the given text, e.g. when the given text has 1-2 bullet points instead of 3-5 points, 
- the irrelevant information (section-level summary that has "I can't summarise...") at the begining or the end in the given text,
- the instruction prompt asks for specific details, e.g. if the prompt for the final summarisation task is changed to "please summarise the given text" rather than the detailed version asking for conclusions and fidnings, the model is able to do so. 

This is not always guaranteed, but the general observation from experiments is that **the model's answer usually depends on the amount and the position of relevant information in the given text. Carefully engineered prompts are still important for the model to handle the given information.**  

In [39]:
section_level_summaries ="\n".join(text_chunks_summary)
# section_level_summaries

In [40]:
paper_summary = summarise_page(section_level_summaries, default_prompts)

'Summary: The research concludes that tree-based models like XGBoost and Random Forests maintain superiority over deep learning methods for handling medium-sized tabular datasets. Significant performance gaps exist even with hyperparameter tuning. This superiority extends beyond handling of categorical variables, as even with numerical features only, the gap persists. \n\nFindings: \n- Tree-based models, including XGBoost and Random Forests, consistently outperform standard and new deep learning methods on medium-sized tabular datasets.\n- The strength of tree-based models persists even after extensive hyperparameter tuning.\n- While previously thought that Neural Networks struggle primarily with categorical variables in tabular data, the study indicated a significant performance gap even when only numerical features were considered.\n\nMethods: The research achieved its findings through comprehensive benchmarking of standard and novel deep learning techniques and tree-based models acr

In [41]:
print(paper_summary)

Summary: The research concludes that tree-based models like XGBoost and Random Forests maintain superiority over deep learning methods for handling medium-sized tabular datasets. Significant performance gaps exist even with hyperparameter tuning. This superiority extends beyond handling of categorical variables, as even with numerical features only, the gap persists. 

Findings: 
- Tree-based models, including XGBoost and Random Forests, consistently outperform standard and new deep learning methods on medium-sized tabular datasets.
- The strength of tree-based models persists even after extensive hyperparameter tuning.
- While previously thought that Neural Networks struggle primarily with categorical variables in tabular data, the study indicated a significant performance gap even when only numerical features were considered.

Methods: The research achieved its findings through comprehensive benchmarking of standard and novel deep learning techniques and tree-based models across many

Useful resources:

1. [A Chunk by Any Other Name: Structured Text Splitting and Metadata-enhanced RAG](https://blog.langchain.dev/a-chunk-by-any-other-name/)
2. [Chunking Strategies for LLM Applications](https://www.pinecone.io/learn/chunking-strategies/)
3. [How to Chunk Text Data — A Comparative Analysis](https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a)
4. [Using Document Layout Structure for Efficient RAG](https://ambikasukla.substack.com/p/efficient-rag-with-document-layout?r=ft8uc&utm_campaign=post&utm_medium=web)